# Key-value Database (Redis)

Author: José Alberto Benítez Andrades - <a href="mailto:jbena@unileon.es">jbena@unileon.es</a><br>

## Reduced wording

There are three parts to the development of the work:
- Implement the design of the database by means of functions that allow the insertion of new data.
- Populate the previously designed database.
- Define functions to carry out tests to check its correct operation.

The details of each section are given below.

### 1. Database design

A number of functions are requested to mimic the operation of Twitter using a key-value database in Redis:

**new_user**: Will receive the name of the new user, generate a new entry using an incremental identifier (provided by Redis) for each user.<br>.
The structure will be of the form user:id where id is the user identifier. In addition, for each user the user name will be stored.<br> <br>
It is also recommended to store all users together with their identifiers in a single structure whose key can be "users".

**Proposed solution**: We propose to generate a hashmap *users* in which we will search if the user already exists. <br>
In case the user does not exist in the dbdd, we will increment a variable one by one that will be used to assign the *id* to the new user, it will be called *user_id*.<br>
In this way we will store in a hashmap called *users* a key-value of the type *username:user_id*.<br>
After this, we will generate a key of type *user:user_id* in which we will store *name:username*, also in a hashmap.
I have chosen to use hashmaps because this time we don't need to get the data in a sorted way at any point.

In [1]:
import redis
import time
import logging

redis_db=redis.Redis(host='127.0.0.1',port=6379, password='', decode_responses=True)

In [2]:
def new_user(username):
    if(redis_db.hget('users',username) == None):
        user_id = redis_db.incr('user_id',1)
        redis_db.hset('users',mapping={username:user_id})
        
        key = "user:"+str(user_id)
        redis_db.hset(key,mapping={"name":username})
    else:
        logging.error(f"User {username} exists.")

**new_follower**: You will receive two usernames. The first one will correspond to a *followed* user and the second one will correspond to the *follower*.<br>
You will also receive a third variable that will correspond to a "timestamp".<br>
It is recommended to use key names of the style *followers:id* where *id* is the identifier of the user being followed.<br> <br>
The structure will contain for each user a set of tuples that will store the *follower* along with a *timestamp* corresponding to the time at which their follower followed them.

**Proposed solution**: After receiving the followed user (*followed*) and the follower (*follower*) we first check if the user name exists in our database.
If it does, the identifiers of both *followed_id* and *follower_id* are obtained.
After this, we check if the follower was already following the followed user using *redis_db.zscore*.<br>
For this occasion, an ordered set of items has been used, as later on it will be requested to receive a list ordered by date of following.

For **new_following** something similar to new_follower is requested but to store the people a user follows. For this reason we choose the same structure as *new_follower*.

And both functions are inserted inside a function called **follow** because we are told that the functions *new_follower* and *new_following* can only be called from *follow*.

In [3]:
def follow(follower,followed,ts):
    def new_follower(followed,follower,timestamp):
        followed_id = redis_db.hget('users',followed)
        if(followed_id != None):
            follower_id = redis_db.hget('users',follower)
            if(follower_id != None):        
                key = "followers:"+followed_id
                if(redis_db.zscore('followers:'+followed_id,follower_id) == None):
                    redis_db.zadd(key,mapping={follower_id:timestamp})
                else:
                    logging.error(f"User {followed} was already followed by {follower}.")
            else:
                logging.error(f"User {follower} doesn't exist.")
        else:
            logging.error(f"User {followed} doesn't exist..")
    def new_following(follower,followed,timestamp):
        follower_id = redis_db.hget('users',follower)
        if(follower_id != None):
            followed_id = redis_db.hget('users',followed)
            if(followed_id != None):
                key = "following:"+follower_id
                if(redis_db.zscore('following:'+follower_id,followed_id) == None):
                    redis_db.zadd(key,mapping={followed_id:timestamp})
                else:
                    logging.error(f"User{follower} was already following {followed}.")
            else:
                logging.error(f"User {followed} doesn't exist.")
        else:
            logging.error(f"User {follower} doesn't exist.")
    new_follower(followed,follower,ts)
    new_following(follower,followed,ts)

**new_post**: This function will be used to add posts/tweets to the database. It will receive as parameters: (i) user who generates the message, (ii) message body and (iii) timestamp with the creation time of the message.<br>.
For each message, a key will be created whose name follows the structure *post:post_id*, where *post_id* will be another incremental counter different from the users' one.
This structure will store, for each post, the id of the user who generates it, the message and the time of creation. But for this, a structure of type *posts:id* will be created where *id* is the user id.
In addition, this list will store not only the posts written by the user himself, but also those written by the people he follows. This would completely simulate the twitter timeline.

**Proposed solution:** This time we first check if the user sending the message exists.
If it does, we increment our post_id variable by one unit.
After that, we store in our *post* hashmap structure the following: post:post_id storing 'sender':sender_id, 'message':message, 'timestamp':timestamp.<br>
On the other hand, a search of the followers is made using *redis_db.zrangebyscore* to which we pass not only the identifier of the one who creates the message, but also the time of creation of the message. This allows us to obtain those folllowers who were actually followers before the user sent the message. In this way we avoid including tweets in the timeline (TL) of users who followed this user after the creation of the message. Although the statement tells us to handle this in the function "get_timeline", I found it much more efficient to do a single check when inserting the tweet. In the other way, we would have to make many more checks, in each TL of each user when calling get_timeline.
After this, we go through each follower and add as a sorted set *posts:follower_id* and inside *post_id:timestamp*.<br>
We also add it in the sender's own TL.
We choose ordered set for *posts* because we will have to get ordered lists later.

In [4]:
def new_post(sender,message,ts):
    sender_id = redis_db.hget('users',sender)
    if(sender_id != None):
        post_id = redis_db.incr('post_id',1)
        redis_db.hset('post:'+str(post_id),mapping={'sender':sender_id,'message':message,'timestamp':ts})
        followers = redis_db.zrangebyscore('followers:'+sender_id,0,ts)
        if(followers != None):
            for key in followers:
                redis_db.zadd('posts:'+str(key),mapping={post_id:ts})
        redis_db.zadd('posts:'+str(sender_id),mapping={post_id:ts})
    else:
        logging.error("The user sending the message does not exist")

In the statement we are told that we can also send as a variable the database where we want to make all these modifications.
In this case, I think it is not necessary to send it as a variable because by modifying the value of redis_db as a global variable, we can also choose which database we want to write to.<br>
It is true that, if we include the db as a parameter of the functions, we could have, for example, declared several databases in several variables and send the one we need at each moment to each function.<br>
However, for the current exercise, I decide to keep it this way in order not to have to send one more parameter in each function, even knowing that it has its advantages if it is a practice in which several databases are used.

Finally, the structure that is formed is as follows:

<img src="images/diagram.jpg" width="700" />

### 2. Data set

In this section we are shown the content of two files, *twitter_sample.csv* and *relations.csv*.<br>
The first file contains 111 tweets with the columns: user, Post_Time and Tweet_Content.<br>
The second file contains 22 relations with the columns: User, Follows and Following_Time.<br>
<br>
One of the steps to be taken to populate the database is to insert all the existing users in both files. To do this we execute the following cells.<br> <br> <br>
First we must load the csv using pandas and we must "clean" those tweets that, for example, do not have a user who sends it (there is one in this list of 111).

In [5]:
import pandas as pd
twsample = pd.read_csv('twitter_sample.csv')
relations = pd.read_csv('relations.csv')

In [6]:
twsample = twsample.drop(['Unnamed: 3'],axis=True)
twsample.dropna(inplace=True)
twsample.head(2)

,User,Post_Time,Tweet_Content
0,andyglittle,13 Jul 2019 05:59:58,We've loved being motivated by the stories of ...
1,andyglittle,05 Jul 2019 10:07:29,Thanks for the shout-out on our #MorethanMedic...


In [7]:
relations.head(2)

,User,Follows,Following_Time
0,roxanefeller,cathcooney,13 Jun 2019 05:59:58
1,andyglittle,charleskod,14 Jul 2019 10:07:29


We convert the dates to timestamp in order to store them in our database.

In [8]:
from datetime import datetime
import numpy as np

twsample['Post_Time'] = pd.to_datetime(twsample['Post_Time'])
twsample['ts'] = twsample.Post_Time.values.astype(np.int64) // 10 ** 9

relations['Following_Time'] = pd.to_datetime(relations['Following_Time'])
relations['ts'] = relations.Following_Time.values.astype(np.int64) // 10 ** 9
relations.head(2)

,User,Follows,Following_Time,ts
0,roxanefeller,cathcooney,2019-06-13 05:59:58,1560405598
1,andyglittle,charleskod,2019-07-14 10:07:29,1563098849


Before populating the database, I leave a commented line here in case you want to delete the current contents of the database.

In [11]:
redis_db.flushdb()

True

We get the unique users to store them all at once.

In [12]:
unique_users = set(list(pd.unique(twsample['User'])) + list(pd.unique(relations['User'])) + list(pd.unique(relations['Follows'])) )

for user in unique_users:
    new_user(user)

This is what the database looks like after populating it with users:

<img src="images/users.png" width="700" />
<img src="images/user-id.png" width="700" />
<img src="images/userid.png" width="700" />

Once the users have been added, we can store the relationships and posts:

In [13]:
for index,row in relations.iterrows():
    follow(row['User'],row['Follows'],row['ts'])

This is what the database looks like after populating it with relationships:

<img src="images/following.png" width="700" />
<img src="images/followers.png" width="700" />

In [15]:
for index, row in twsample.iterrows():
    new_post(row['User'],row['Tweet_Content'],row['ts'])

This is what the database looks like after populating it with relationships:

<img src="images/posts-id.png" width="700" />
<img src="images/post.png" width="700" />
<img src="images/postid.png" width="700" />

Fully populated, the following numbers are observed in each hashmap, sorted set and counters:

<img src="images/poblada.png" width="150" />

### 3.Testing the databases

To finish this task, the functions **obtain_followers**, **obtain_followings** and **obtain_timeline** must be created to verify if the system is working correctly.
<br>
Among the elements to take into account in these functions, we have the following:
- **get_followers**: It will receive a user name and will return or print all the user names that follow him/her and when they started following him/her, but in date format, ordered in time.
- get_followings**: A function analogous to the previous one but with the users a user follows.
- get_timeline**: This function receives a user name and a boolean that determines whether we want to show the user's own tweets or not.

In [16]:
'''
    This function receives the user name in followed.
    It searches for the user's identifier.
    If it exists, it searches for users using the zrange function, since we chose to store them as a sorted set.
    Thanks to this it is possible to obtain the sorted users, in this case, from newest to oldest user.
    The statement did not indicate whether it had to be like this, or in reverse order.
    After obtaining this list, I store it in a list, followers_l, which I can then display when calling the function.
    To display it correctly, it is recommended to call the function in the following way:
    
    pd.DataFrame(get_followers("animalhealthEU"))
    
    It should be noted that the timestamp had to be converted to date using datetime.utcfromtimestamp.
    Without utc there was a 2 hour time offset.
'''
def get_followers(followed):
    followed_id = redis_db.hget('users',followed)
    if(followed_id != None):
        followers = redis_db.zrange('followers:'+str(followed_id),0,-1,desc=True,withscores=True)
        followers_l = []
        for follower in followers:
            follower_l = []
            follower_l.append(redis_db.hget('user:'+str(follower[0]),'name'))
            follower_l.append(datetime.utcfromtimestamp(int(follower[1])).strftime("%d %b %Y %H:%M:%S"))
            followers_l.append(follower_l)
        return followers_l
    else:
        logging.error("El usuario no existe")

In [17]:
'''
    In this case, this function works exactly the same as the previous one, only to receive the users that a user follows.
    It is recommended to call this function as follows:
    
    pd.DataFrame(get_followings("animalhealthEU"))
'''
def get_followings(follower):
    follower_id = redis_db.hget('users',follower)
    if(follower_id != None):
        followings = redis_db.zrange('following:'+str(follower_id),0,-1,desc=True,withscores=True)
        followings_l = []
        for followed in followings:
            followed_l = []
            followed_l.append(redis_db.hget('user:'+str(followed[0]),'name'))
            followed_l.append(datetime.utcfromtimestamp(int(followed[1])).strftime("%d %b %Y %H:%M:%S"))
            followings_l.append(followed_l)
        return followings_l
    else:
        logging.error("El usuario no existe")

In [18]:
'''
    Finally, get_timeline has been programmed.
    We get the username from which we get the id.
    If the user exists, then we search for tweets using sort.
    Thanks to sort we get the tweets sorted and, in addition, we do as if it were a "join" with post: we get all the values of each tweet directly (the date, the date, the time, the time of the tweet).
    directly all the values of each tweet (the date, the person who sent it and the message).
    We have obtained the tweets in descending order on the understanding that on Twitter the newest tweets are seen first.
    
    After having obtained the tweets, I check:
        - If Tweets_owns is false and the user sending the tweet I scroll through is different from the user I get the TL from, I show it or if tweets_owns is true, too.
        This way I control whether to show tweets from the person I get the TL from or not.
    
    Going through each tweet:
        - I generate a list, tweet_list, in which I collect the values of the 'message', 'timestamp' and 'sender' keys.
        - I add the elements to the list in order: user, message, date.
    
    
    It is recommended to call this function in the following way:
    
    pd.DataFrame(get_timeline("animalhealthEU",True))
    pd.DataFrame(get_timeline("animalhealthEU",False))
'''
def get_timeline(username,own_tweets):
    user_id = redis_db.hget('users',username)
    if(user_id != None):
        #tweets = redis_db.zrange('posts:'+str(user_id),0,-1,desc=False)
        # Como he utilizado un conjunto ordenado, obtengo la misma salida si uso sort que si uso zrange
        tweets = redis_db.sort("posts:"+str(user_id), by="post:*->timestamp",get=["post:*->timestamp","post:*->sender","post:*->message"], groups=True, desc=True)
        timeline = []
        for tweet in tweets:
            if((own_tweets == False and user_id != tweet[1]) or own_tweets):
                tweet_list = []
                msg = tweet[2]
                ts = tweet[0]
                sender_id = tweet[1]
                sender_name = redis_db.hget('user:'+str(sender_id),'name')
                date = datetime.utcfromtimestamp(int(ts)).strftime("%d %b %Y %H:%M:%S")
                tweet_list.append(sender_name)
                tweet_list.append(msg)
                tweet_list.append(date)
                timeline.append(tweet_list)
        return timeline
    else:
        logging.error("El usuario no existe")

Algunas pruebas realizadas con el conjunto de datos utilizado:

In [19]:
pd.DataFrame(get_followers("animalhealthEU"))

,0,1
0,alkhalilkouma,01 Jul 2019 19:25:03
1,dkalnow,01 Jul 2019 08:40:26
2,charleskod,01 Jul 2019 07:27:58
3,afparron,01 Jun 2019 06:04:40


In [21]:
pd.DataFrame(get_followings("animalhealthEU"))

,0,1
0,dkalnow,27 Aug 2019 13:57:04
1,drshahrul80,19 Jul 2019 14:59:55


In [ ]:
pd.DataFrame(get_followers("karin_stowell"))

In [ ]:
pd.DataFrame(get_followings("karin_stowell"))

In [ ]:
pd.DataFrame(get_timeline("andyglittle",True))

In [ ]:
pd.DataFrame(get_timeline("afparron",True))

In [ ]:
pd.DataFrame(get_timeline("drshahrul80",True))

In [ ]:
pd.DataFrame(get_timeline("karin_stowell",True))

In [ ]:
pd.DataFrame(get_timeline("cathcooney",True))

In [ ]:
pd.DataFrame(get_timeline("dkalnow",True))

In [ ]:
pd.DataFrame(get_timeline("alkhalilkouma",True))

In [ ]:
pd.DataFrame(get_timeline("seers_helen",True))

In [ ]:
pd.DataFrame(get_timeline("seers_helen",False))

In [ ]:
pd.DataFrame(get_timeline("hanyshita",True))

In [ ]:
pd.DataFrame(get_timeline("hanyshita",False))

In [ ]:
pd.DataFrame(get_timeline("roxanefeller",True))

In [ ]:
pd.DataFrame(get_timeline("animalhealthEU",True))

In [ ]:
pd.DataFrame(get_timeline("charleskod",True))